In [1]:
from italian_csv_type_prediction.models.tree import generate_training_set
from sklearn.ensemble import RandomForestClassifier
import compress_pickle

In [2]:
X, Y = generate_training_set(subsets_number=10000)

In [3]:
tree = RandomForestClassifier(n_estimators=500, max_depth=10)
tree.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
(tree.predict(X) == Y).mean()

In [ ]:
compress_pickle.dump(tree, "italian_csv_type_prediction/models/tree.pkl.gz")